#### Imports

In [8]:
import os
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
nltk.download('vader_lexicon')
from scipy.special import softmax
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, XLNetTokenizer, TFXLNetForSequenceClassification

## Filtering 3-Star Reviews

In [10]:
# Load the combined reviews data from the saved file
combined_reviews_df = pd.read_csv('processed_data/combined_reviews.csv', encoding='utf-8', engine='python')

# Filter for reviews with rating = 3 to perform sentiment analysis on the same
three_stars = combined_reviews_df[combined_reviews_df['rating'] == 3]

#print(filtered_df.head())
print(f"Total reviews with rating of 3: {len(three_stars)}")

Total reviews with rating of 3: 8055




---



## Sentiment Analysis on Product Reviews

### xlnet-base-cased-product-review-sentiment-analysis

Below we use [xlnet-base-cased-product-review-sentiment-analysis](https://huggingface.co/dipawidia/xlnet-base-cased-product-review-sentiment-analysis)model from Hugging face to perform sentiment analysis.

In [15]:
# Initialize the tokenizer and model
tokenizer = XLNetTokenizer.from_pretrained("dipawidia/xlnet-base-cased-product-review-sentiment-analysis")
model = TFXLNetForSequenceClassification.from_pretrained("dipawidia/xlnet-base-cased-product-review-sentiment-analysis")

# Function to analyze sentiment for a batch of reviews
def get_sentiment_batch(reviews_batch):
    tokenize_text = tokenizer(reviews_batch.tolist(), padding=True, truncation=True, return_tensors='tf', max_length=512)
    preds = model.predict(dict(tokenize_text))['logits']
    class_preds = np.argmax(tf.keras.layers.Softmax()(preds), axis=-1)
    labels = ['Positive' if pred == 1 else 'Negative' for pred in class_preds]
    return labels

# Batch size for processing reviews
batch_size = 64

# Initialize the progress bar for batching
num_batches = len(three_stars) // batch_size + (1 if len(three_stars) % batch_size != 0 else 0)

# List to hold sentiment results
sentiment_results = []

# Process reviews in batches
for i in tqdm(range(0, len(three_stars), batch_size), total=num_batches, desc="Processing reviews", unit="batch"):
    batch = three_stars['review_text'][i:i + batch_size]  # Get the current batch of reviews
    batch_sentiments = get_sentiment_batch(batch)  # Get sentiments for the current batch
    sentiment_results.extend(batch_sentiments)  # Append results

# Assign the sentiment results back to the dataframe
three_stars['sentiment'] = sentiment_results

# Count positive and negative sentiments
sentiment_counts = three_stars['sentiment'].value_counts()
print(sentiment_counts)

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFXLNetForSequenceClassification.

All the weights of TFXLNetForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetForSequenceClassification for predictions without further training.
Processing reviews:   0%|          | 0/1012 [00:00<?, ?batch/s]

2/2 [==============================] - 4s 720ms/step


Processing reviews:   0%|          | 1/1012 [00:05<1:31:18,  5.42s/batch]

2/2 [==============================] - 3s 554ms/step


Processing reviews:   0%|          | 2/1012 [00:09<1:19:31,  4.72s/batch]

...

Processing reviews: 100%|█████████▉| 1011/1012 [1:23:46<00:06,  6.82s/batch]

1/1 [==============================] - 1s 807ms/step


Processing reviews: 100%|██████████| 1012/1012 [1:23:48<00:00,  4.97s/batch]

sentiment
Negative    41791
Positive    22945
Name: count, dtype: int64



<ipython-input-15-c280ebfe8f1c>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  three_stars['sentiment'] = sentiment_results


In [16]:
# Display a few positive sentiment examples
positive_sentiments = three_stars[three_stars['sentiment'] == 'Positive']
print("Positive Sentiment Examples:")
print(positive_sentiments[['review_text']])

Positive Sentiment Examples:
                                              review_text
14      It’s super glossy and looks amazing but doesn’...
40      This product is marketed as a sleeping mask bu...
53      I think this is overpriced for what it is - li...
92      Meh, I mean it’s ok. I like the Fenty skin one...
112     I used to love this product. Because it feels ...
...                                                   ...
782943  is alright nothing special leaves skin soft ea...
782960  Saw all the AB hype and decided to give the br...
782966  Very moisturizing and feels really nice, but i...
783006  Pros:-soothing and moisturizing - it feels lik...
783076  Been using it for a month but sadly didn’t any...

[22945 rows x 1 columns]


In [17]:
# Display a few negative sentiment examples
negative_sentiments = three_stars[three_stars['sentiment'] == 'Negative']
print("Negative Sentiment Examples:")
print(negative_sentiments[['review_text']])

Negative Sentiment Examples:
                                              review_text
11      Does the gummy bear mask smell absolutely deli...
30      ive tried my friend’s a few times and it does ...
60      I have a sample size of this product. It is ve...
64      I don’t see the hype for this product. It does...
80      I tried it first from the mini version Sephora...
...                                                   ...
782983  This is a thick moisturizing cream, and compar...
782987  So so. Sticky, smells weird. Not much help to ...
782988  I love the texture. It layers really nicely wi...
782994  I paid a fortune for this cream but I’m not co...
783075  Im heading into my 30s so I wanted to splurge ...

[41791 rows x 1 columns]


### Get all positive and negative reviews


In [18]:
positive_reviews = three_stars[three_stars['sentiment'] == 'Positive']
negative_reviews = three_stars[three_stars['sentiment'] == 'Negative']

positive_path = 'processed_data/positive_reviews.csv'
positive_reviews.to_csv(positive_path, index=False)

negative_path = 'processed_data/negative_reviews.csv'
negative_reviews.to_csv(negative_path, index=False)



---



## SentimentIntensityAnalyzer

We also tried to compute sentiment analysis with [SentimentIntensityAnalyzer](https://www.nltk.org/howto/sentiment.html) - one of the most useful NLTK (Natural Language Toolkit) features. It is a pre-trained model that can be used to perform sentiment analysis on text data.

In [4]:
sia = SentimentIntensityAnalyzer()

def get_sentiment(row):
    # Get the sentiment scores
    scores = sia.polarity_scores(row['review_text'])

    # Decide sentiment based on compound score
    if scores['compound'] >= 0.05:
        return 'Positive'
    elif scores['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

three_stars['sentiment'] = three_stars.apply(get_sentiment, axis=1)

# Count positives and negatives
sentiment_counts = three_stars['sentiment'].value_counts()
print(sentiment_counts)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


sentiment
Positive    6690
Negative    1044
Neutral      321
Name: count, dtype: int64


<ipython-input-4-ac0ceac9cdac>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  three_stars['sentiment'] = three_stars.apply(get_sentiment, axis=1)


### Get all positive and negative reviews


In [7]:
# Display a few positive sentiment examples
positive_sentiments = three_stars[three_stars['sentiment'] == 'Positive']
print("Positive Sentiment Examples:")
print(positive_sentiments[['review_text']])

Positive Sentiment Examples:
                                              review_text
11      Does the gummy bear mask smell absolutely deli...
14      It’s super glossy and looks amazing but doesn’...
30      ive tried my friend’s a few times and it does ...
40      This product is marketed as a sleeping mask bu...
53      I think this is overpriced for what it is - li...
...                                                   ...
102089  I recently started using this multi action vit...
102114  Love that this is a moisturizer and spf in one...
102125  I really wanted to love this product! It made ...
102142  This Flower Acid serum interested me from the ...
102180  This is a great and affordable spf option. I l...

[6690 rows x 1 columns]


In [6]:
# Display a few negative sentiment examples
negative_sentiments = three_stars[three_stars['sentiment'] == 'Negative']
print("Negative Sentiment Examples:")
print(negative_sentiments[['review_text']])

Negative Sentiment Examples:
                                              review_text
113     I would not repurchase as there are better one...
122     I like this product overall, I find that moist...
277     I have mixed feelings about the product. Yes i...
493     the main reason i bought this was for the scen...
679     Had to try Mango, but it’s scent/ flavour is t...
...                                                   ...
101520  I’ve been using it since the first week of Aug...
101722  Use IOP and Tan-Luxe products and while I want...
101846  If you like tacky, utlra-high gloss lip produc...
101984  Was nothing amazing, pretty much just a typica...
102078  I did see a difference my problem is my eyes b...

[1044 rows x 1 columns]




---



## cardiffnlp/twitter-xlm-roberta-base-sentiment

In [ ]:
# Model path for the new model
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = TFAutoModelForSequenceClassification.from_pretrained(model_path)

# Function to analyze sentiment for a batch of reviews
def get_sentiment_batch(reviews_batch):
    # Tokenize the batch of reviews
    tokenize_text = tokenizer(reviews_batch.tolist(), padding=True, truncation=True, return_tensors='tf', max_length=512)

    # Get predictions from the model
    preds = model(**tokenize_text)['logits']

    # Apply softmax to get class probabilities
    scores = softmax(preds.numpy(), axis=-1)

    # Get the class with the highest probability (0 = negative, 2 = positive)
    class_preds = np.argmax(scores, axis=-1)

    # Assign sentiment labels based on the class prediction
    labels = ['Negative' if pred == 0 else 'Neutral' if pred == 1 else 'Positive' for pred in class_preds]

    return labels

# Batch size for processing reviews
batch_size = 64

# Ensure the reviews are valid strings and filter out NaN values
three_stars['review_text'] = three_stars['review_text'].fillna('').astype(str)

# Initialize the progress bar for batching
num_batches = len(three_stars) // batch_size + (1 if len(three_stars) % batch_size != 0 else 0)

# List to hold sentiment results
sentiment_results = []

# Process reviews in batches
for i in tqdm(range(0, len(three_stars), batch_size), total=num_batches, desc="Processing reviews", unit="batch"):
    batch = three_stars['review_text'][i:i + batch_size]  # Get the current batch of reviews
    batch_sentiments = get_sentiment_batch(batch)  # Get sentiments for the current batch
    sentiment_results.extend(batch_sentiments)  # Append results

# Assign the sentiment results back to the dataframe
three_stars['sentiment'] = sentiment_results

# Count positive, negative, and neutral sentiments
sentiment_counts = three_stars['sentiment'].value_counts()
print(sentiment_counts)

### Get all positive and negative reviews


In [ ]:
# Display a few positive sentiment examples
positive_sentiments = three_stars[three_stars['sentiment'] == 'Positive']
print("Positive Sentiment Examples:")
print(positive_sentiments[['review_text']])

In [ ]:
# Display a few negative sentiment examples
negative_sentiments = three_stars[three_stars['sentiment'] == 'Negative']
print("Negative Sentiment Examples:")
print(negative_sentiments[['review_text']])